# Solar Wind Tools Run All
In this notebook we will use all the functions currently available in Solar Wind Tools to create our solar wind dataset

## Downloading
First we import the download functon.\
Then provide a file path and the start year and we end year.\
We can also provide the month we start at in the start year and the month we end at in the end year\

In this exampe we are downloading from 2000 to 2001 and including the full year for the start and end year. We then save it into a file called 'omni_1min.h5'

In [2]:
from Solar_Wind_Tools.download import download_omni_1min
download_omni_1min(fromYear=2000, toYear=2001, monthFirstYear=1, monthLastYear=12, path='./omni_1min.h5')


--2024-09-12 17:32:37--  https://cdaweb.gsfc.nasa.gov/sp_phys/data/omni/hro_1min/2000/omni_hro_1min_20000101_v01.cdf
Resolving cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)... 2001:4d0:2418:121::65, 169.154.154.65
Connecting to cdaweb.gsfc.nasa.gov (cdaweb.gsfc.nasa.gov)|2001:4d0:2418:121::65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8769714 (8.4M) [application/x-cdf]
Saving to: ‘omni_hro_1min_20000101_v01.cdf’

     0K .......... .......... .......... .......... ..........  0%  259K 33s
    50K .......... .......... .......... .......... ..........  1%  504K 25s
   100K .......... .......... .......... .......... ..........  1% 1.13M 19s
   150K .......... .......... .......... .......... ..........  2%  318M 14s
   200K .......... .......... .......... .......... ..........  2%  833K 13s
   250K .......... .......... .......... .......... ..........  3% 1.13M 12s
   300K .......... .......... .......... .......... ..........  4% 72.8M 10s
   350K .......

The hdf key for the omni data is 'omni'

## Calculate Statistics
We are now going to calculate statistics using a 10 minute window.\
We provide the file path of the omni data. We set the save path as the same so that it uses the same file.\
The load key is the hdf file key for the omni data.
The key is the key the new dataset is saved to that has the statistics. We have set it to 'omni_window'

In [ ]:
from Solar_Wind_Tools.window import statistics
statistics(file_path='./omni_1min.h5', save_path='./omni_1min.h5', window=10, load_key='omni', key='omni_window')

In the 'omni_1min.h5' file we now have two keys omni with the original data and omni_window which has the original data and the statistics
## Coupling Functions
So far only the newell coupling function has been implemented. We will calculate its value and calculate the mean across the 10 minute window.
If a window is not provided then the mean is not calculated

In [ ]:
from Solar_Wind_Tools.window import coupling
coupling('./omni_1min.h5', './omni_1min.h5', window=10, load_key='omni_window', key='omni_window')


## Dipole Tilt
We use the package at available at: https://github.com/klaundal/dipole' to calculate the dipole tilt.\

We use the same load and save file and alter the same hdf key.


In [ ]:
from Solar_Wind_Tools.window import dipole
dipole('./omni_1min.h5', './omni_1min.h5', load_key='omni_window', key='omni_window')

## Time Shifting
This useful if you want to shift the time to create a delay or lag between the solar wind data and the data you are trying to associate it with.
time shift is in minutes we have you used 10 minutes. Positive value will make a time step be associated with a value from 10 minutes ago.\
This shift is only applied to statistics columns.

In [ ]:
from Solar_Wind_Tools.window import time_shift
time_shift('./omni_1min.h5', './omni_1min.h5', shift=10, load_key='omni_window', key='omni_window')


## Shortcut to running multiple
We have provided a run all function to do a one line running of all functions.\
You provide a list of strings for all the functions you want to run. The options are: ['Download', 'Statistics', 'Coupling', 'Dipole', 'Time Shift'].\

Here we download from 2000 to 2001. We calculate the statistics using a 2 hour window. We then shift the statistics by 20 minutes.

In [ ]:
from Solar_Wind_Tools.run_all import run_all
run_all('./test_omni.hdf5',
        key='omni_window_2h_20', window=2*60, shift=20, start_year= 2000, 
        end_year=2001, run=['Download', 'Statistics', 'Coupling', 'Dipole', 'Time Shift'])

In [ ]:
import pandas as pd
pd.read_hdf('./test_omni.hdf5', key='omni')

## We use HDF files because they are searchable without loading it all into ram

In [ ]:
pd.read_hdf('./test_omni.hdf5', key='omni_window_2h_20', where="index>='2001-01-01T00:00'")